In [21]:
pip install twilio


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------------------------- ------ 1.6/1.9 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


Armando URL

In [34]:
query = 'Bogotá'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f3cf65ff7ae64bccad9210657220807&q=Bogotá&days=1&aqi=no&alerts=no'

In [36]:
response = requests.get(url_clima).json()

In [38]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [40]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [42]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [44]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1737435600,
 'time': '2025-01-21 00:00',
 'temp_c': 9.3,
 'temp_f': 48.7,
 'is_day': 0,
 'condition': {'text': 'Patchy rain nearby',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 1.6,
 'wind_kph': 2.5,
 'wind_degree': 146,
 'wind_dir': 'SSE',
 'pressure_mb': 1019.0,
 'pressure_in': 30.08,
 'precip_mm': 0.05,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 88,
 'cloud': 61,
 'feelslike_c': 9.3,
 'feelslike_f': 48.7,
 'windchill_c': 9.3,
 'windchill_f': 48.7,
 'heatindex_c': 9.3,
 'heatindex_f': 48.7,
 'dewpoint_c': 7.4,
 'dewpoint_f': 45.2,
 'will_it_rain': 0,
 'chance_of_rain': 66,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 2.8,
 'gust_kph': 4.5,
 'uv': 0}

In [46]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2025-01-21'

In [62]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [54]:
response['forecast']['forecastday'][0]['hour'][0]['condition']

{'text': 'Patchy rain nearby',
 'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
 'code': 1063}

In [56]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

9.3

In [58]:
response['forecast']['forecastday'][0]['hour'][2]['will_it_rain']

1

In [60]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

100

In [157]:
def get_forecast(response, i):
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])  # corregido aquí
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]

    return {
        "hora": hora,
        "condicion": condicion,
        "temperatura": tempe,
        "lluvia": rain,
        "probabilidad_lluvia": prob_rain,
        "fecha": fecha
    }


In [90]:
from tqdm import tqdm
import pandas as pd

In [159]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='green'):
    datos.append(get_forecast(response, i))

# Convertir la lista de diccionarios en un DataFrame
df = pd.DataFrame(datos)

# Mostrar el DataFrame para verificar datos
print(df.head())
    

100%|███████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 63310.25it/s]

   hora           condicion  temperatura  lluvia  probabilidad_lluvia  \
0     0  Patchy rain nearby          9.3       0                   66   
1     1  Patchy rain nearby          9.2       1                   89   
2     2  Patchy rain nearby          8.6       1                  100   
3     3  Patchy rain nearby          8.3       1                   86   
4     4      Partly Cloudy           8.0       0                    0   

        fecha  
0  2025-01-21  
1  2025-01-21  
2  2025-01-21  
3  2025-01-21  
4  2025-01-21  


In [161]:
datos[0]

{'hora': 0,
 'condicion': 'Patchy rain nearby',
 'temperatura': 9.3,
 'lluvia': 0,
 'probabilidad_lluvia': 66,
 'fecha': '2025-01-21'}

In [165]:
col = ['hora', 'condicion', 'temperatura', 'lluvia', 'probabilidad_lluvia','fecha']
df = pd.DataFrame(datos, columns=col)
df = df.sort_values(by='hora', ascending=True)
df

,hora,condicion,temperatura,lluvia,probabilidad_lluvia,fecha
0,0,Patchy rain nearby,9.3,0,66,2025-01-21
1,1,Patchy rain nearby,9.2,1,89,2025-01-21
2,2,Patchy rain nearby,8.6,1,100,2025-01-21
3,3,Patchy rain nearby,8.3,1,86,2025-01-21
4,4,Partly Cloudy,8.0,0,0,2025-01-21
5,5,Partly Cloudy,7.5,0,0,2025-01-21
6,6,Partly cloudy,9.2,0,0,2025-01-21
7,7,Partly Cloudy,8.6,0,0,2025-01-21
8,8,Sunny,12.0,0,0,2025-01-21
9,9,Patchy rain nearby,14.4,0,63,2025-01-21


In [167]:
df_rain =  df[(df['lluvia']==1) & (df['hora']>6) & (df['hora']< 22)]
df_rain = df_rain[['hora','condicion']]
df_rain.set_index('hora', inplace = True)

In [169]:
df_rain

,condicion
hora,
10,Patchy rain nearby
11,Patchy rain nearby
12,Patchy rain nearby
13,Patchy rain nearby
14,Patchy rain nearby
15,Patchy rain nearby
16,Patchy rain nearby
17,Patchy rain nearby
19,Patchy rain nearby


In [171]:
df['fecha'][0]

'2025-01-21'

In [177]:
PHONE_NUMBER

'+1 507 570 5538'

In [233]:
'Hola! \n\n\n El pronóstico del tiempo hoy ' + df['fecha'][0] + ' en ' + query + ' es: \n\n\n ' + str(df_rain)

'Hola! \n\n\n El pronóstico del tiempo hoy 2025-01-21 en Bogotá es: \n\n\n                condicion\nhora                    \n10    Patchy rain nearby\n11    Patchy rain nearby\n12    Patchy rain nearby\n13    Patchy rain nearby\n14    Patchy rain nearby\n15    Patchy rain nearby\n16    Patchy rain nearby\n17    Patchy rain nearby\n19    Patchy rain nearby'

In [3]:
import os

# Definir variables de entorno dentro del script
os.environ["TWILIO_ACCOUNT_SID"] = "AC20e094a129cd1f75e253f9a1e6d95e38"
os.environ["TWILIO_AUTH_TOKEN"] = "a9f18df2bc8a0934a0eb8b499e6b113e"
os.environ["PHONE_NUMBER"] = "+15075705538"

# Luego puedes acceder a ellas con getenv
account_sid = os.getenv("TWILIO_ACCOUNT_SID")
auth_token = os.getenv("TWILIO_AUTH_TOKEN")
phone_number = os.getenv("PHONE_NUMBER")

print("SID:", account_sid)
print("Phone:", phone_number)

NameError: name 'time' is not defined

In [261]:
import os
import time
from twilio.rest import Client

# Esperar 2 segundos antes de enviar el mensaje (opcional)
time.sleep(2)

# Obtener las credenciales de Twilio desde las variables de entorno
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
phone_number = os.getenv('PHONE_NUMBER')

# Verificar que las variables no sean None para evitar errores
if not account_sid or not auth_token or not phone_number:
    raise ValueError("Error: Las variables de entorno de Twilio no están configuradas correctamente.")

# Inicializar el cliente de Twilio
client = Client(account_sid, auth_token)

# Construcción del mensaje
try:
    message = client.messages.create(
        body='\nHola! \n\n\n El pronóstico de lluvia hoy ' + df['fecha'][0] + 
             ' en ' + query + ' es : \n\n\n ' + str(df_rain),
        from_=phone_number,
        to='+34604180382'
    )

    print('Mensaje Enviado: ' + message.sid)

except Exception as e:
    print("Error al enviar el mensaje:", str(e))



Mensaje Enviado: SM8ea08d7dcaae8662bb00b82586f859e1


SID: AC20e094a129cd1f75e253f9a1e6d95e38
Phone: +15075705538
